In [2]:
from langgraph.graph import StateGraph, START , END
from typing import TypedDict , Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from dotenv import load_dotenv

# Tools node and Tool conditions
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool  # for Custom tool

import requests
import random

In [ ]:
# Creating Tools
''' In Langchain 2 types of Tool (i) Prebuilt  (ii) Custom:Created By user and defined with @tool decorator'''

search_tool = DuckDuckGoSearchRun(region="us-en")  #prebuilt Tool

#Custom tool
@tool
def calculator(first_num: float , second_num: float , operation: str) ->dict:
    """
    Perform a basic airthmetic Operation on Two Numbers.
    Supported Operations: add,sub,mul,div.
    """
    try:
        if operation == "add":
            result = first_num+second_num
        elif operation == "sub":
            result = first_num-second_num
        elif operation == "mul":
            result = first_num*second_num
        elif operation == "div":
            if second_num == 0:
                return {'error' : 'Division by Zero is not allowed'}
            result = first_num/second_num
        else:
            return {"error" : f"unsupported Operation '{operation}' "}

        return {"first_num ":first_num ,"second_num":second_num,"operation":operation,"result":result}
    
    except Exception as e:
        return {'error':str(e)}
    
@tool
def get_stock_price(symbol: str) ->dict:
    """ Fetch the latest stock price for a given symbol (eg. 'AAPL' ,'TSLA') 
        using Alpha vintage With Api key in the URl.
    """
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey=C9PE94QUEW9VWGFM"
    r= requests.get(url)
    return r.json()



In [5]:
llm = ChatOpenAI()

In [ ]:
# Make tool List
tool= [get_stock_price,search_tool,calculator]

# Make the LLM tool aware
llm_with_tools = llm.bind_tools(tools)

In [4]:
# state
class ChatState(TypedDict):
    messages:Annotated[list[BaseMessage],add_messages]

In [ ]:
# Graph Node Functions

# Chat Node
def chat_node(state: ChatState):
    """LLM node that may Answer or request a tool call."""
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    return {"messages" :[response]}

# Tool Node
tool_node=ToolNode(tools) # Execute tool call

In [ ]:
# graph define
graph = StateGraph(ChatState)

# Nodes
graph.add_node('chat_node',chat_node)
graph.add_node('tool_node',tool_node)

# Edges
graph.add_edge(START,'chat_node')
# If LLM ask for a tool go to ToolNode; else Finish
graph.add_conditional_edges("chat_node",tools_condition)

# Compile graph
chatbot = graph.compile()


In [ ]:
# Regular Chat
out = chatbot.invoke({'messages':[HumanMessage(content ="hello")]})
print(out['messages'][-1].content)

In [ ]:
# Chat requiring tool
out = chatbot.invoke({'messages':[HumanMessage(content ="what is stock price of apple")]})
print(out['messages'][-1].content)

Problem is Now ?

(i) Output of Tools is usually not a refines so we can directly 
display to user , but our desgined workflow is after going to 
tool node directly go to end node , not refining a output.

(ii) For MultiStep Tool usage Involvement Query (like using 
tool 2 time in single query) result is not coming , since after 
going first time from chatNode to ToolNode it is directly going 
to End Node

example : Multi Step Tool Query

What is the stock price of apple today and how much it cost if 
buy a 70 stock?

1st Step : Stock Tool Use: Find Price

2nd Step : Calculator Tool use: Multiply Stock price with 70


Now Our WorkFlow is Modified from
instead of directly Going from (i) chatNode to End and From (ii) ChatNode to {ToolNode to End}

Now From (i) ToolNode Going Back to ChatNode and (ii) from chatNode to End





Remaining Code is Same Only Workflow code changed


In [ ]:
# graph define
graph = StateGraph(ChatState)

# Nodes
graph.add_node('chat_node',chat_node)
graph.add_node('tool_node',tool_node)

# Edges
graph.add_edge(START,'chat_node')
# If LLM ask for a tool go to ToolNode; else Finish
graph.add_conditional_edges("chat_node",tools_condition)
# Changed add
graph.add_edge("tools" , "chat_node")

# Compile graph
chatbot = graph.compile()
